Colab for https://github.com/Mozer/YuE-extend
- It can extend any uploaded mp3
- uses exllama2 module
- flash_attention 2 is not supported for Tesla T4

# Install

In [3]:
!pip install exllamav2
!git clone https://github.com/Mozer/YuE-extend
%cd YuE-extend
!pip install -r requirements.txt
!git clone https://huggingface.co/m-a-p/xcodec_mini_infer
!pip install --upgrade protobuf!pip uninstall -y flash-attn

fatal: destination path 'YuE-extend' already exists and is not an empty directory.
/content/YuE-extend
fatal: destination path 'xcodec_mini_infer' already exists and is not an empty directory.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -y


# Download models

In [1]:
# download COT s1 + s2 models
# you can also download ICL model (optional)
!huggingface-cli download Alissonerdx/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw --local-dir workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw
!huggingface-cli download Alissonerdx/YuE-s2-1B-general-exl2-8.0bpw --local-dir workspace/models/YuE-s2-1B-general-exl2-8.0bpw

Fetching 7 files:   0% 0/7 [00:00<?, ?it/s]Downloading 'model.safetensors.index.json' to 'workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw/.cache/huggingface/download/yVzAsSxRSINSz-tQbpx-TLpfkLU=.c16ffa0c29ff3c8c70c6248a1027989584facb1f.incomplete'

tokenizer.model: 100% 1.76M/1.76M [00:00<00:00, 83.2MB/s]
Download complete. Moving file to workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw/tokenizer.model

output.safetensors:   0% 0.00/5.79G [00:00<?, ?B/s]Downloading 'generation_config.json' to 'workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw/.cache/huggingface/download/3EVKVggOldJcKSsGjSdoUCN1AyQ=.59c0f3c6815a220b6b4e852c51be873503df2ce0.incomplete'

output.safetensors:   1% 31.5M/5.79G [00:00<00:20, 275MB/s]

model.safetensors.index.json: 100% 23.9k/23.9k [00:00<00:00, 59.9MB/s]
Download complete. Moving file to workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw/model.safetensors.index.json


README.md: 100% 6.87k/6.87k [00:00<00:00, 38.2MB/s]
Download complete. Moving 

Don't start UI. Google colab doesn't like UIs. Use command line instead. You can edit lyrics and genre in Files on the left -> Yue-extend/prompt_egs/lyrics.txt (double click). Output mp3 will be saved in workspace/outputs/.

# lyrics2music

In [4]:
# simple cot music generation. 1 verse (run_n_segments), 30s (max_new_tokens)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 30000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 3000 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --resume_after_n -1

Loading exllamav2_ext extension (JIT)...
Building C++/CUDA extension ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:15:17 0:00:00
 ## Warning: Flash Attention is installed but unsupported GPUs were detected.
Starting stage 1...
 ## Warning: Flash Attention is installed but unsupported GPUs were detected.
2025-04-07 20:55:41.260448: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744059341.566327    6362 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744059341.652768    6362 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 20:55:42.292436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to

# mp3 extend

Trim your mp3 up to 30s. Use "Files" menu on the left to upload your split pop.00001.Vocals.mp3 + pop.00001.Instrumental.mp3 to Yue-extend/prompt_egs. To split use: https://huggingface.co/spaces/theneos/audio-separator or other services.
Don't forget to update genre.txt and put full lyrics into lyrics.txt

In [ ]:
# mp3 extend cot. 1 verse (run_n_segments), 28s (max_new_tokens), 7s from mp3 (extend_mp3_end_time)
!python -u src/yue/infer.py --stage1_use_exl2 --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" --stage1_cache_size 25000 --stage1_cache_mode FP16 --stage2_use_exl2 --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" --stage2_cache_size 50000 --stage2_cache_mode 'FP16' --genre_txt "prompt_egs/genre.txt" --lyrics_txt "prompt_egs/lyrics.txt" --run_n_segments 1 --output_dir workspace/outputs --cuda_idx 0 --seed 42 --max_new_tokens 2800 --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth --config_path xcodec_mini_infer/decoders/config.yaml --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth --extend_mp3 --vocal_track_prompt_path "prompt_egs/pop.00001.Vocals.mp3" --instrumental_track_prompt_path "prompt_egs/pop.00001.Instrumental.mp3" --extend_mp3_end_time 7 --prompt_start_time 0 --prompt_end_time 8 --resume_after_n -1

In [ ]:
!python -u src/yue/infer.py \
  --stage1_use_exl2 \
  --stage1_model "workspace/models/YuE-s1-7B-anneal-en-cot-exl2-8.0bpw" \
  --stage1_cache_size 25000 \
  --stage1_cache_mode FP16 \
  --stage2_use_exl2 \
  --stage2_model "workspace/models/YuE-s2-1B-general-exl2-8.0bpw" \
  --stage2_cache_size 50000 \
  --stage2_cache_mode FP16 \
  --genre_txt "prompt_egs/genre.txt" \
  --lyrics_txt "prompt_egs/lyrics.txt" \
  --run_n_segments 1 \
  --output_dir workspace/outputs \
  --cuda_idx 0 \
  --seed 42 \
  --max_new_tokens 2800 \
  --basic_model_config xcodec_mini_infer/final_ckpt/config.yaml \
  --resume_path xcodec_mini_infer/final_ckpt/ckpt_00360000.pth \
  --config_path xcodec_mini_infer/decoders/config.yaml \
  --vocal_decoder_path xcodec_mini_infer/decoders/decoder_131000.pth \
  --inst_decoder_path xcodec_mini_infer/decoders/decoder_151000.pth \
  --extend_mp3 \
  --instrumental_track_prompt_path "/content/segment_5.wav" \
  --extend_mp3_end_time 7 \
  --prompt_start_time 0 \
  --prompt_end_time 8 \
  --resume_after_n -1